In [1]:
%matplotlib notebook
import flux

Missing flux value for: GLEAM J202818-254302 at frequency: 204 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 212 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 220 MHz.
Missing flux value for: GLEAM J202818-254302 at frequency: 227 MHz.
Missing spectral index for: GLEAM J202818-254302


//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:18: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.pos1_Elm_samples = h5f["pos1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:19: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.neg1_Elm_samples = h5f["neg1_Elm"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:20: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.freq_nodes = h5f["frequencies"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:21: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  self.L_model = h5f["spatial_bandlimit"].value
//anaconda3/envs/hera/lib/python3.7/site-packages/spin1_beam_model/jones_matrix_field.py:22: H5pyDeprec

delta_nu_in is 1000000.0


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import healpy as hp

In [3]:
arbitrary_source = flux.catalog.obj_catalog[999]
arb_ra = arbitrary_source.ra_angle
arb_dec = arbitrary_source.dec_angle
J = flux.stokes.J_matrix(arb_ra, arb_dec)
J.shape

(1, 2, 2)

In [4]:
arb_ra = flux.rot.get_lst()
arb_dec = np.radians(flux.rot.hera_lat)
J = flux.stokes.J_matrix(arb_ra, arb_dec)
J

array([[[ 2.77629327e-05+3.19202085e-05j,
          8.12625503e-01-5.82786230e-01j],
        [ 8.12625503e-01-5.82786230e-01j,
         -2.77629312e-05-3.19202064e-05j]]])

In [5]:
Am = flux.stokes.A_matrix(arb_ra, arb_dec)
print(Am)
Am.shape

[[[ 1.61751813e-06+4.21190303e-05j  5.00021866e-01-5.00020249e-01j
    4.99979747e-01+4.99978130e-01j  1.77662902e-09-4.86592307e-15j]]

 [[ 4.99979747e-01+4.99978130e-01j -6.10356695e-15-1.11424294e-15j
    4.21171962e-05-1.61751724e-06j -4.99978130e-01+4.99979747e-01j]]

 [[ 5.00021866e-01-5.00020249e-01j -4.21207495e-05+1.61751724e-06j
    3.68363448e-15+1.16960183e-15j -5.00020249e-01-5.00021866e-01j]]

 [[ 1.77662673e-09+4.92128514e-15j -5.00020249e-01-5.00021866e-01j
   -4.99978130e-01+4.99979747e-01j -1.61751813e-06-4.21190303e-05j]]]


(4, 1, 4)

In [6]:
Am[0][0][0]

(1.6175181290238645e-06+4.211903028594271e-05j)

In [12]:
nside = 128
hp.mollview(np.real(Am[:, 0, 0]), rot=[0, 90], title='xy? real')

TypeError: bad number of pixels

In [7]:
# Display available antennae
print(flux.demo.active_ants)

[0, 1, 2, 11, 12, 13, 14, 23, 24, 25, 26, 27, 36, 37, 38, 39, 40, 41, 50, 51, 52, 53, 54, 55, 65, 66, 67, 68, 69, 70, 71, 82, 83, 84, 85, 86, 87, 88, 98, 120, 121, 122, 123, 124, 136, 137, 138, 139, 140, 141, 142, 143]


In [8]:
# we sample three sources almost at random, and 2/3 are zero

print(arbitrary_source)
print(flux.vis.visibility(2, 11, arbitrary_source))

print(flux.vis.visibility(36, 37, flux.catalog.obj_catalog[981]))
print(flux.vis.visibility(88, 121, flux.catalog.obj_catalog[134]))
# also, would we not expect the first term to dominate? And yet it is not even the largest
    # of numbers all with magnitudes less than one

Name: GLEAM J185915-381842
Right ascension: 284.8155
Declination: -37.688250000000004
151 MHz flux: 1.2316479999999999

[[-2.52345024e-05+5.66182527e-05j]
 [-2.80554445e-04-4.88635152e-04j]
 [ 6.68504215e-04-1.56137891e-04j]
 [ 2.99442453e-04-1.82246566e-04j]]
[[ 6.92156535e-04+8.28480729e-05j]
 [ 2.41565795e-03+1.02088328e-03j]
 [ 1.59820895e-03-5.66043987e-03j]
 [-4.97599238e-06-2.70555676e-04j]]
[[0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]


In [9]:
"""
This section kills the kernel! Do NOT run it!!
I am leaving it here as a to-do of sorts. I think, that my inability
to run it in this environment may be a bad sign.
(It does work fine in the shell, so far as I can tell.)

t = 0
arg = 134

obj = flux.catalog.obj_catalog[arg]
ra = np.radians(obj.ra_angle)
dec = np.radians(obj.dec_angle)
while t < 2 * np.pi:
    print(t)
    t += np.pi / 72
    
    print(flux.stokes.J_matrix(ra, dec, t))
    print("\n")

"""

'\nThis section kills the kernel! Do NOT run it!!\nI am leaving it here as a to-do of sorts. I think, that my inability\nto run it in this environment may be a bad sign.\n(It does work fine in the shell, so far as I can tell.)\n\nt = 0\narg = 134\n\nobj = flux.catalog.obj_catalog[arg]\nra = np.radians(obj.ra_angle)\ndec = np.radians(obj.dec_angle)\nwhile t < 2 * np.pi:\n    print(t)\n    t += np.pi / 72\n    \n    print(flux.stokes.J_matrix(ra, dec, t))\n    print("\n")\n\n'

In [14]:
# Split request -> the following sections belong in the multi-source notebook

# test visibility over time function
# we are expecting an array of 4x1 complex vectors
vist = flux.vis.visibility_over_time(88, 121, 0, 2 / 3 * np.pi, np.pi / 72)

AttributeError: module 'flux.vis' has no attribute 'visibility_over_time'

In [ ]:
flux.vis.